In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv(r'C:\Cypher Analytica\PROJECT\Amazon Project\Amazon_sales.csv')


In [75]:
df["Payment_Method"].value_counts()

Payment_Method
Cash on Delivery    3827
Credit Card         3800
Debit Card          3727
UPI                 3646
Name: count, dtype: int64

In [4]:
df.head(10)

,Order_ID,Date,Customer_ID,Product_Category,Product_Name,Quantity,Unit_Price_INR,Total_Sales_INR,Payment_Method,Delivery_Status,Review_Rating,Review_Text,State,Country
0,ORD100000,2025-01-25,CUST2796,Home & Kitchen,Cookware Set,2,25574.41,51148.82,Credit Card,Returned,1,Waste of money,Sikkim,India
1,ORD100001,2025-08-28,CUST9669,Beauty,Hair Dryer,1,19361.41,19361.41,Debit Card,Returned,5,Excellent product!,Telangana,India
2,ORD100002,2025-02-27,CUST5808,Electronics,Tablet,3,38476.22,115428.66,Cash on Delivery,Delivered,3,Fair deal,Nagaland,India
3,ORD100003,2025-02-24,CUST5889,Electronics,Headphones,5,38145.72,190728.60,Credit Card,Delivered,5,Highly recommend!,Assam,India
4,ORD100004,2025-06-15,CUST9005,Clothing,Saree,5,45940.98,229704.90,UPI,Delivered,5,Highly recommend!,Odisha,India
5,ORD100005,2025-02-10,CUST2403,Home & Kitchen,Cookware Set,1,12195.41,12195.41,Cash on Delivery,Delivered,5,Fantastic quality!,Maharashtra,India
6,ORD100006,2025-05-16,CUST9645,Clothing,Jeans,4,47353.06,189412.24,Cash on Delivery,Delivered,2,Poor packaging,Arunachal Pradesh,India
7,ORD100007,2025-08-13,CUST9479,Clothing,Sneakers,4,49758.44,199033.76,Cash on Delivery,Delivered,3,Okay product,Andhra Pradesh,India
8,ORD100008,2025-04-28,CUST4608,Clothing,T-Shirt,3,1247.51,3742.53,UPI,Delivered,5,Excellent product!,Assam,India
9,ORD100009,2025-09-21,CUST4899,Clothing,T-Shirt,1,43009.84,43009.84,Debit Card,Returned,1,Terrible experience,Gujarat,India


# Business Problem Statements 


## 1. 	Which product categories and products contribute most to revenue?


In [5]:
# Product Category
prod_cat = df.groupby("Product_Category", as_index = False)["Total_Sales_INR"].sum()

In [6]:
pd.options.display.float_format = '{:,.2f}'.format
prod_cat

,Product_Category,Total_Sales_INR
0,Beauty,"227,489,624.68"
1,Books,"224,999,226.56"
2,Clothing,"222,409,335.83"
3,Electronics,"226,564,923.33"
4,Home & Kitchen,"216,698,693.10"


In [7]:
prod_cat_sorted = prod_cat.sort_values(by='Total_Sales_INR',ascending=False).reset_index()
prod_cat_sorted = prod_cat_sorted.drop(columns='index')

In [8]:
top_category = prod_cat.loc[prod_cat['Total_Sales_INR'].idxmax()]
print(top_category)

Product_Category           Beauty
Total_Sales_INR    227,489,624.68
Name: 0, dtype: object


In [9]:
top_category_percentage = (
    top_category['Total_Sales_INR'] / df["Total_Sales_INR"].sum()
) * 100

In [10]:
print(
    f"Top Category: {top_category['Product_Category']}\n"
    f"Contribution to Total Sales: {top_category_percentage:.2f}%"
)

Top Category: Beauty
Contribution to Total Sales: 20.34%


In [11]:
# Individual Product
product = df.groupby("Product_Name", as_index = False)["Total_Sales_INR"].sum()
product = product.sort_values(by = "Total_Sales_INR",ascending = False)

In [12]:
product_reset = product.reset_index(drop = True)

In [13]:
top_product = product.loc[product['Total_Sales_INR'].idxmax()]
print(top_product)

Product_Name           Lipstick
Total_Sales_INR   48,159,020.15
Name: 12, dtype: object


In [14]:
top_product_percentage = (
    top_product['Total_Sales_INR'] / df["Total_Sales_INR"].sum()
) * 100

In [15]:
print(
    f"Top Product: {top_product['Product_Name']}\n"
    f"Contribution to Total Sales: {top_product_percentage:.2f}%"
)

Top Product: Lipstick
Contribution to Total Sales: 4.31%


## 2. •	How do returns and delivery status impact overall sales performance?


In [16]:
sales_by_status = df.groupby("Delivery_Status")["Total_Sales_INR"].sum().reset_index()
sales_by_status

,Delivery_Status,Total_Sales_INR
0,Delivered,"378,749,035.26"
1,Pending,"376,214,132.94"
2,Returned,"363,198,635.30"


In [17]:
total_orders = len(df)
returned_orders = len(df[df["Delivery_Status"] == "Returned"])

return_rate = (returned_orders / total_orders) * 100
print(f"The total Return Rate is : {return_rate}% ")

The total Return Rate is : 32.54% 


In [18]:
returned_sales_loss = df[df["Delivery_Status"] == "Returned"]["Total_Sales_INR"].sum()
print(f"The total Loss due to retun is : {round(returned_sales_loss,2)} Rs")

The total Loss due to retun is : 363198635.3 Rs


In [19]:
impact_summary = df.groupby("Delivery_Status").agg(
    Total_Orders=("Order_ID", "count"),
    Total_Revenue=("Total_Sales_INR", "sum"),
    Avg_Order_Value=("Total_Sales_INR", "mean")
).reset_index()

impact_summary

,Delivery_Status,Total_Orders,Total_Revenue,Avg_Order_Value
0,Delivered,5075,"378,749,035.26","74,630.35"
1,Pending,5044,"376,214,132.94","74,586.47"
2,Returned,4881,"363,198,635.30","74,410.70"


In [20]:
returns_by_category = df[df["Delivery_Status"] == "Returned"] .groupby("Product_Category", as_index = False)["Order_ID"].count().sort_values(by = "Product_Category",ascending=False)

returns_by_category


,Product_Category,Order_ID
4,Home & Kitchen,910
3,Electronics,1006
2,Clothing,982
1,Books,1029
0,Beauty,954


In [21]:
returns_rating = df[df["Delivery_Status"] == "Returned"] \
    .groupby("Review_Rating", as_index = False)["Order_ID"].count()

returns_rating

,Review_Rating,Order_ID
0,1,929
1,2,920
2,3,996
3,4,990
4,5,1046


### •	What is the relationship between customer ratings, reviews, and product returns?

In [22]:
df.groupby("Review_Rating")["Delivery_Status"] \
  .value_counts(normalize=True) * 100

Review_Rating  Delivery_Status
1              Delivered         35.05
               Returned          32.72
               Pending           32.23
2              Delivered         35.12
               Pending           34.19
               Returned          30.69
3              Pending           33.59
               Delivered         33.36
               Returned          33.06
4              Pending           34.05
               Delivered         33.19
               Returned          32.76
5              Pending           33.98
               Returned          33.44
               Delivered         32.58
Name: proportion, dtype: float64

### •	Which states generate the highest sales and order volumes?

In [23]:
state_sales = (df.groupby('State')['Total_Sales_INR'].sum().sort_values(ascending=False).reset_index(name='Total_Sales_INR'))

state_sales.head(5)

,State,Total_Sales_INR
0,Sikkim,"43,113,469.51"
1,Rajasthan,"42,906,175.08"
2,Chhattisgarh,"42,857,545.27"
3,Meghalaya,"42,773,152.96"
4,Tamil Nadu,"41,967,968.99"


In [24]:
state_orders = (df.groupby('State')['Order_ID'].nunique().sort_values(ascending=False).reset_index(name='Order_Count'))

state_orders.head(5)

,State,Order_Count
0,Sikkim,596
1,Rajasthan,568
2,Tamil Nadu,567
3,Meghalaya,559
4,Chhattisgarh,556


In [25]:
state_performance = (df.groupby('State').agg(
          Total_Sales_INR=('Total_Sales_INR', 'sum'),
          Order_Count=('Order_ID', 'nunique')
                                              ).reset_index().sort_values('Total_Sales_INR', ascending=False)
)

state_performance.head(5)

,State,Total_Sales_INR,Order_Count
21,Sikkim,"43,113,469.51",596
20,Rajasthan,"42,906,175.08",568
4,Chhattisgarh,"42,857,545.27",556
15,Meghalaya,"42,773,152.96",559
22,Tamil Nadu,"41,967,968.99",567


### •	How do different payment methods affect order completion and returns?

In [26]:
payment_status = (df.groupby(['Payment_Method', 'Delivery_Status'])['Order_ID'].nunique().reset_index(name='Order_Count'))

payment_status

,Payment_Method,Delivery_Status,Order_Count
0,Cash on Delivery,Delivered,1329
1,Cash on Delivery,Pending,1274
2,Cash on Delivery,Returned,1224
3,Credit Card,Delivered,1244
4,Credit Card,Pending,1289
5,Credit Card,Returned,1267
6,Debit Card,Delivered,1287
7,Debit Card,Pending,1221
8,Debit Card,Returned,1219
9,UPI,Delivered,1215


In [27]:
# MAking a Pivot table for better readablity 

In [28]:
payment_pivot = payment_status.pivot(
    index='Payment_Method',
    columns='Delivery_Status',
    values='Order_Count'
).fillna(0)

payment_pivot

Delivery_Status,Delivered,Pending,Returned
Payment_Method,,,
Cash on Delivery,1329,1274,1224
Credit Card,1244,1289,1267
Debit Card,1287,1221,1219
UPI,1215,1260,1171


In [29]:
payment_pivot['Total_Orders'] = payment_pivot.sum(axis=1)

payment_pivot['Return_Rate_%'] = (payment_pivot.get('Returned', 0) / payment_pivot['Total_Orders']) * 100
payment_pivot

Delivery_Status,Delivered,Pending,Returned,Total_Orders,Return_Rate_%
Payment_Method,,,,,
Cash on Delivery,1329,1274,1224,3827,31.98
Credit Card,1244,1289,1267,3800,33.34
Debit Card,1287,1221,1219,3727,32.71
UPI,1215,1260,1171,3646,32.12


In [30]:
payment_pivot = payment_pivot.sort_values('Return_Rate_%', ascending=False)
payment_pivot

Delivery_Status,Delivered,Pending,Returned,Total_Orders,Return_Rate_%
Payment_Method,,,,,
Credit Card,1244,1289,1267,3800,33.34
Debit Card,1287,1221,1219,3727,32.71
UPI,1215,1260,1171,3646,32.12
Cash on Delivery,1329,1274,1224,3827,31.98


### Basic Analysis

# 1.	Analyze sales performance across product categories and products.


In [31]:
category_sales = (df.groupby('Product_Category')['Total_Sales_INR'].sum().sort_values(ascending=False).reset_index())

category_sales

,Product_Category,Total_Sales_INR
0,Beauty,"227,489,624.68"
1,Electronics,"226,564,923.33"
2,Books,"224,999,226.56"
3,Clothing,"222,409,335.83"
4,Home & Kitchen,"216,698,693.10"


In [32]:
category_orders = (df.groupby('Product_Category')['Order_ID'].nunique().reset_index(name='Order_Count'))

In [33]:
category_performance = category_sales.merge(category_orders,on='Product_Category')

category_performance



,Product_Category,Total_Sales_INR,Order_Count
0,Beauty,"227,489,624.68",2997
1,Electronics,"226,564,923.33",3036
2,Books,"224,999,226.56",3035
3,Clothing,"222,409,335.83",3022
4,Home & Kitchen,"216,698,693.10",2910


In [34]:
top_category = category_sales.iloc[0]
top_category

Product_Category           Beauty
Total_Sales_INR    227,489,624.68
Name: 0, dtype: object

In [35]:
lowest_category = category_sales.iloc[-1]
lowest_category

Product_Category    Home & Kitchen
Total_Sales_INR     216,698,693.10
Name: 4, dtype: object

In [36]:
category_sales['Sales_Percentage'] = (
    category_sales['Total_Sales_INR'] / category_sales['Total_Sales_INR'].sum()
) * 100

category_sales

,Product_Category,Total_Sales_INR,Sales_Percentage
0,Beauty,"227,489,624.68",20.34
1,Electronics,"226,564,923.33",20.26
2,Books,"224,999,226.56",20.12
3,Clothing,"222,409,335.83",19.89
4,Home & Kitchen,"216,698,693.10",19.38


In [37]:
products_sales = (df.groupby('Product_Name')['Total_Sales_INR'].sum().sort_values(ascending=False).reset_index())

products_sales

,Product_Name,Total_Sales_INR
0,Lipstick,"48,159,020.15"
1,Children's Book,"48,145,664.52"
2,Headphones,"48,044,405.75"
3,Hair Dryer,"47,428,844.81"
4,Perfume,"46,284,732.62"
5,Laptop,"46,173,868.15"
6,Tablet,"46,170,022.15"
7,Sneakers,"46,057,295.93"
8,Air Fryer,"45,919,707.91"
9,Jeans,"45,232,819.47"


In [38]:
product_orders = (df.groupby('Product_Name')['Order_ID'].nunique().reset_index(name='Order_Count'))

In [39]:
product_performance = products_sales.merge(product_orders,on='Product_Name')

product_performance

,Product_Name,Total_Sales_INR,Order_Count
0,Lipstick,"48,159,020.15",590
1,Children's Book,"48,145,664.52",636
2,Headphones,"48,044,405.75",610
3,Hair Dryer,"47,428,844.81",599
4,Perfume,"46,284,732.62",629
5,Laptop,"46,173,868.15",618
6,Tablet,"46,170,022.15",616
7,Sneakers,"46,057,295.93",596
8,Air Fryer,"45,919,707.91",614
9,Jeans,"45,232,819.47",604


In [40]:
top_product = products_sales.iloc[0]
top_product

Product_Name           Lipstick
Total_Sales_INR   48,159,020.15
Name: 0, dtype: object

In [41]:
lowest_product = products_sales.iloc[-1]
lowest_product

Product_Name       Vacuum Cleaner
Total_Sales_INR     41,229,356.61
Name: 24, dtype: object

In [42]:
products_sales['Sales_Percentage'] = (
    products_sales['Total_Sales_INR'] / products_sales['Total_Sales_INR'].sum()
) * 100

products_sales

,Product_Name,Total_Sales_INR,Sales_Percentage
0,Lipstick,"48,159,020.15",4.31
1,Children's Book,"48,145,664.52",4.31
2,Headphones,"48,044,405.75",4.30
3,Hair Dryer,"47,428,844.81",4.24
4,Perfume,"46,284,732.62",4.14
5,Laptop,"46,173,868.15",4.13
6,Tablet,"46,170,022.15",4.13
7,Sneakers,"46,057,295.93",4.12
8,Air Fryer,"45,919,707.91",4.11
9,Jeans,"45,232,819.47",4.05


### 2.	Identify revenue-driving and underperforming segments.

In [43]:
category_revenue = category_sales.copy()

In [44]:
category_revenue = category_revenue.sort_values(by='Total_Sales_INR',ascending=False)
category_revenue

,Product_Category,Total_Sales_INR,Sales_Percentage
0,Beauty,"227,489,624.68",20.34
1,Electronics,"226,564,923.33",20.26
2,Books,"224,999,226.56",20.12
3,Clothing,"222,409,335.83",19.89
4,Home & Kitchen,"216,698,693.10",19.38


In [45]:
revenue_drivers = category_revenue[category_revenue['Sales_Percentage'] >= 20]

revenue_drivers

,Product_Category,Total_Sales_INR,Sales_Percentage
0,Beauty,"227,489,624.68",20.34
1,Electronics,"226,564,923.33",20.26
2,Books,"224,999,226.56",20.12


In [46]:
onderperforming_categories = category_revenue[category_revenue['Sales_Percentage'] <= 20]

onderperforming_categories

,Product_Category,Total_Sales_INR,Sales_Percentage
3,Clothing,"222,409,335.83",19.89
4,Home & Kitchen,"216,698,693.10",19.38


In [47]:
# Relation between Total Sales and Order Count per category 

category_orders = (df.groupby('Product_Category')['Order_ID'].nunique().reset_index(name='Order_Count'))

category_performance = category_revenue.merge(category_orders,on='Product_Category')

category_performance

,Product_Category,Total_Sales_INR,Sales_Percentage,Order_Count
0,Beauty,"227,489,624.68",20.34,2997
1,Electronics,"226,564,923.33",20.26,3036
2,Books,"224,999,226.56",20.12,3035
3,Clothing,"222,409,335.83",19.89,3022
4,Home & Kitchen,"216,698,693.10",19.38,2910


# 3.	Evaluate customer behavior using review ratings and feedback.

In [48]:
rating_distribution = df['Review_Rating'].value_counts().sort_index()
rating_distribution

Review_Rating
1    2839
2    2998
3    3013
4    3022
5    3128
Name: count, dtype: int64

In [49]:
avg_rating_category = (df.groupby('Product_Category')['Review_Rating'].mean().reset_index().sort_values('Review_Rating', ascending=False))

avg_rating_category

,Product_Category,Review_Rating
3,Electronics,3.07
4,Home & Kitchen,3.06
0,Beauty,3.04
1,Books,3.03
2,Clothing,3.00


In [50]:
df['Review_Type'] = pd.cut(df['Review_Rating'],bins=[0, 2, 3, 5],labels=['Negative', 'Neutral', 'Positive'])

In [51]:
review_count = df['Review_Type'].value_counts()
review_count

Review_Type
Positive    6150
Negative    5837
Neutral     3013
Name: count, dtype: int64

In [52]:
review_returns = (df.groupby(['Review_Type', 'Delivery_Status'])['Customer_ID'].nunique().reset_index(name='Order_Count'))

review_returns

,Review_Type,Delivery_Status,Order_Count
0,Negative,Delivered,1832
1,Negative,Pending,1721
2,Negative,Returned,1679
3,Neutral,Delivered,960
4,Neutral,Pending,951
5,Neutral,Returned,943
6,Positive,Delivered,1823
7,Positive,Pending,1879
8,Positive,Returned,1820


In [53]:
returned_only = review_returns[review_returns["Delivery_Status"] == "Returned"]
returned_only

,Review_Type,Delivery_Status,Order_Count
2,Negative,Returned,1679
5,Neutral,Returned,943
8,Positive,Returned,1820


In [54]:
return_rate = (df.groupby('Review_Type')['Delivery_Status'].apply(lambda x: (x == 'Returned').mean() * 100).reset_index(name='Return_Rate_%'))

return_rate


,Review_Type,Return_Rate_%
0,Negative,31.68
1,Neutral,33.06
2,Positive,33.11


In [55]:
# Category with most negative reviews
negative_feedback = (df[df['Review_Type'] == 'Negative'].groupby('Product_Category').size().reset_index(name='Negative_Review_Count').sort_values('Negative_Review_Count', ascending=False))

negative_feedback

,Product_Category,Negative_Review_Count
2,Clothing,1221
1,Books,1177
0,Beauty,1172
3,Electronics,1166
4,Home & Kitchen,1101


In [56]:
delivery_distribution = (df['Delivery_Status'].value_counts().reset_index(name='Order_Count'))

delivery_distribution

,Delivery_Status,Order_Count
0,Delivered,5075
1,Pending,5044
2,Returned,4881


In [57]:
total_orders = df['Customer_ID'].nunique()

returned_orders = df[df['Delivery_Status'] == 'Returned']['Customer_ID'].nunique()

overall_return_rate = round((returned_orders / total_orders) * 100,2)

overall_return_rate

51.32

In [58]:
category_delivery = (df.groupby(['Product_Category', 'Delivery_Status'])['Customer_ID'].nunique().reset_index(name='Order_Count'))

In [59]:
category_delivery_pivot = category_delivery.pivot(index='Product_Category',columns='Delivery_Status',values='Order_Count').fillna(0)

category_delivery_pivot

Delivery_Status,Delivered,Pending,Returned
Product_Category,,,
Beauty,1005,918,910
Books,958,936,968
Clothing,936,983,935
Electronics,955,961,950
Home & Kitchen,939,956,866


In [60]:
# REturn Rate Percentage

category_delivery_pivot['Total_Orders'] = category_delivery_pivot.sum(axis=1)

category_delivery_pivot['Return_Rate_%'] = (category_delivery_pivot.get('Returned', 0) /category_delivery_pivot['Total_Orders']) * 100

category_delivery_pivot.sort_values('Return_Rate_%', ascending=False)

Delivery_Status,Delivered,Pending,Returned,Total_Orders,Return_Rate_%
Product_Category,,,,,
Books,958,936,968,2862,33.82
Electronics,955,961,950,2866,33.15
Clothing,936,983,935,2854,32.76
Beauty,1005,918,910,2833,32.12
Home & Kitchen,939,956,866,2761,31.37


In [61]:
# State wise return rate 
state_return_rate = (
    df.groupby('State')['Delivery_Status']
      .apply(lambda x: (x == 'Returned').mean() * 100)
      .reset_index(name='Return_Rate_%')
      .sort_values('Return_Rate_%', ascending=False)
)

state_return_rate.reset_index(drop = True)

,State,Return_Rate_%
0,Assam,36.35
1,Uttar Pradesh,35.57
2,Nagaland,34.31
3,West Bengal,34.09
4,Bihar,33.88
5,Manipur,33.87
6,Karnataka,33.77
7,Tamil Nadu,33.69
8,Punjab,33.59
9,Meghalaya,33.45


### 5.	Understand geographic trends in sales across Indian states.

In [62]:
# Total Sales based on the States

state_sales = (df.groupby('State')['Total_Sales_INR'].sum().reset_index().sort_values('Total_Sales_INR', ascending=False))

state_sales

,State,Total_Sales_INR
21,Sikkim,"43,113,469.51"
20,Rajasthan,"42,906,175.08"
4,Chhattisgarh,"42,857,545.27"
15,Meghalaya,"42,773,152.96"
22,Tamil Nadu,"41,967,968.99"
25,Uttar Pradesh,"41,690,917.07"
3,Bihar,"41,669,240.44"
27,West Bengal,"41,195,932.45"
24,Tripura,"41,103,376.81"
18,Odisha,"40,924,381.38"


In [63]:
## Order Value per State
state_orders = (df.groupby('State')['Customer_ID'].nunique().reset_index(name='Order_Count').sort_values('Order_Count', ascending=False)
)

state_orders

,State,Order_Count
21,Sikkim,572
20,Rajasthan,550
22,Tamil Nadu,546
12,Madhya Pradesh,541
14,Manipur,541
2,Assam,540
15,Meghalaya,540
3,Bihar,538
9,Jharkhand,535
17,Nagaland,533


In [64]:
# Combining sales and order count 

state_performance = (df.groupby('State').agg(
         Total_Sales_INR=('Total_Sales_INR', 'sum'),
          Order_Count=('Order_ID', 'nunique'),
          Customer_Count=('Customer_ID', 'nunique')
      )
      .reset_index()
)

In [65]:
state_performance['Avg_Order_Value'] = (
    state_performance['Total_Sales_INR'] /
    state_performance['Order_Count']
)

state_performance = state_performance.sort_values('Total_Sales_INR', ascending=False)

state_performance

,State,Total_Sales_INR,Order_Count,Customer_Count,Avg_Order_Value
21,Sikkim,"43,113,469.51",596,572,"72,338.04"
20,Rajasthan,"42,906,175.08",568,550,"75,539.04"
4,Chhattisgarh,"42,857,545.27",556,532,"77,081.92"
15,Meghalaya,"42,773,152.96",559,540,"76,517.27"
22,Tamil Nadu,"41,967,968.99",567,546,"74,017.58"
25,Uttar Pradesh,"41,690,917.07",551,528,"75,664.10"
3,Bihar,"41,669,240.44",549,538,"75,900.26"
27,West Bengal,"41,195,932.45",528,515,"78,022.60"
24,Tripura,"41,103,376.81",540,523,"76,117.36"
18,Odisha,"40,924,381.38",542,523,"75,506.24"


In [66]:
top_states = state_performance.head(5)
bottom_states = state_performance.tail(5)
print(f" The Top 5 perfoming states are : \n",top_states)
print(f"\n The bottom 5 perfoming states are : \n",bottom_states)

 The Top 5 perfoming states are : 
            State  Total_Sales_INR  Order_Count  Customer_Count  \
21        Sikkim    43,113,469.51          596             572   
20     Rajasthan    42,906,175.08          568             550   
4   Chhattisgarh    42,857,545.27          556             532   
15     Meghalaya    42,773,152.96          559             540   
22    Tamil Nadu    41,967,968.99          567             546   

    Avg_Order_Value  
21        72,338.04  
20        75,539.04  
4         77,081.92  
15        76,517.27  
22        74,017.58  

 The bottom 5 perfoming states are : 
                State  Total_Sales_INR  Order_Count  Customer_Count  \
10         Karnataka    38,444,302.71          539             513   
8   Himachal Pradesh    38,182,171.87          492             477   
26       Uttarakhand    37,633,617.48          503             493   
0     Andhra Pradesh    36,237,577.25          480             462   
6            Gujarat    36,226,185.91        

## 6.	Study payment method preferences and their impact on successful deliveries

In [67]:
# Payment Prefrrence Count
payment_preference = (df.groupby('Payment_Method')['Order_ID'].nunique().reset_index(name='Order_Count').sort_values('Order_Count', ascending=False))

payment_preference

,Payment_Method,Order_Count
0,Cash on Delivery,3827
1,Credit Card,3800
2,Debit Card,3727
3,UPI,3646


In [68]:
# Payment With respect to Delivery Status 
payment_delivery = (df.groupby(['Payment_Method', 'Delivery_Status'])['Order_ID'].nunique().reset_index(name='Order_Count'))

payment_delivery

,Payment_Method,Delivery_Status,Order_Count
0,Cash on Delivery,Delivered,1329
1,Cash on Delivery,Pending,1274
2,Cash on Delivery,Returned,1224
3,Credit Card,Delivered,1244
4,Credit Card,Pending,1289
5,Credit Card,Returned,1267
6,Debit Card,Delivered,1287
7,Debit Card,Pending,1221
8,Debit Card,Returned,1219
9,UPI,Delivered,1215


In [69]:
# Pivot for payment vs delivery status
payment_delivery_pivot = payment_delivery.pivot(
    index='Payment_Method',
    columns='Delivery_Status',
    values='Order_Count'
).fillna(0)

payment_delivery_pivot

Delivery_Status,Delivered,Pending,Returned
Payment_Method,,,
Cash on Delivery,1329,1274,1224
Credit Card,1244,1289,1267
Debit Card,1287,1221,1219
UPI,1215,1260,1171


In [70]:
#Delivery sucess rate and return rate 
payment_delivery_pivot['Total_Orders'] = payment_delivery_pivot.sum(axis=1)

payment_delivery_pivot['Delivery_Success_%'] = (
    payment_delivery_pivot.get('Delivered', 0) /
    payment_delivery_pivot['Total_Orders']
) * 100

payment_delivery_pivot['Return_Rate_%'] = (
    payment_delivery_pivot.get('Returned', 0) /
    payment_delivery_pivot['Total_Orders']
) * 100

payment_delivery_pivot = payment_delivery_pivot.sort_values(
    'Delivery_Success_%', ascending=False
)

payment_delivery_pivot

Delivery_Status,Delivered,Pending,Returned,Total_Orders,Delivery_Success_%,Return_Rate_%
Payment_Method,,,,,,
Cash on Delivery,1329,1274,1224,3827,34.73,31.98
Debit Card,1287,1221,1219,3727,34.53,32.71
UPI,1215,1260,1171,3646,33.32,32.12
Credit Card,1244,1289,1267,3800,32.74,33.34


In [71]:
pd.to_excel("")

AttributeError: module 'pandas' has no attribute 'to_excel'